In [379]:
import dataclasses
from collections import Counter
from typing import Tuple, List

from protein import trypsin
from measurement import read_mgf, PeptideMeasurement
from pyteomics import mass


@dataclasses.dataclass
class Modification:
    name: str
    mass: float
    count: int


@dataclasses.dataclass
class Peptide:
    beginning: int
    end: int
    seq: str
    modifications: List[Modification]

    _aas = None
    _mass = None
    _minmass = None
    _maxmass = None

    def count(self, amino_acid):
        if self._aas is None:
            self._aas = Counter(self.seq)
        return self._aas[amino_acid]

    @property
    def zwitterion_mass(self):
        if self._mass is None:
            self._mass = mass.calculate_mass(
                sequence=self.seq, ion_type="M", charge=0
            ) - mass.calculate_mass(formula="H2O")
        return self._mass

    @property
    def min_mass(self):
        if self._minmass is None:
            neg = sum(m.mass * m.count for m in self.modifications if m.mass < 0)
            self._minmass = self.zwitterion_mass + neg
        return self._minmass

    @property
    def max_mass(self):
        if self._maxmass is None:
            pos = sum(m.mass * m.count for m in self.modifications if m.mass > 0)
            self._maxmass = self.zwitterion_mass + pos
        return self._maxmass


In [390]:
LYS = "KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL"

BSA = "DTHKSEIAHRFKDLGEEHFKGLVLIAFSQYLQQCPFDEHVKLVNELTEFAKTCVADESHAGCEKSLHTLFGDELCKVASLRETYGDMADCCEKQEPERNECFLSHKDDSPDLPKLKPDPNTLCDEFKADEKKFWGKYLYEIARRHPYFYAPELLYYANKYNGVFQECCQAEDKGACLLPKIETMREKVLTSSARQRLRCASIQKFGERALKAWSVARLSQKFPKAEFVEVTKLVTDLTKVHKECCHGDLLECADDRADLAKYICDNQDTISSKLKECCDKPLLEKSHCIAEVEKDAIPENLPPLTADFAEDKDVCKNYQEAKDAFLGSFLYEYSRRHPEYAVSVLLRLAKEYEATLEECCAKDDPHACYSTVFDKLKHLVDEPQNLIKQNCDQFEKLGEYGFQNALIVRYTRKVPQVSTPTLVEVSRSLGKVGTRCCTKPESERMPCTEDYLSLILNRLCVLHEKTPVSEKVTKCCTESLVNRRPCFSALTPDETYVPKAFDEKLFTFHADICTLPDTEKQIKKQTALVELLKHKPKATEEQLKTVMENFVAFVDKCCAADDKEACFAVEGPKLVVSTQTALA"

measurements = {m.scan: m for m in read_mgf("../data/mgf/190318_LYS_AT_50x_05.mgf")}

In [392]:
peptides = []
for b, e in trypsin(LYS):
    seq = LYS[b:e]
    met_ox = Modification("met_ox", 15.9949, sum(aa == "M" for aa in seq))
    peptides.append(Peptide(b, e, seq, [met_ox]))

peptides

[Peptide(beginning=0, end=1, seq='K', modifications=[Modification(name='met_ox', mass=15.9949, count=0)]),
 Peptide(beginning=1, end=5, seq='VFGR', modifications=[Modification(name='met_ox', mass=15.9949, count=0)]),
 Peptide(beginning=5, end=13, seq='CELAAAMK', modifications=[Modification(name='met_ox', mass=15.9949, count=1)]),
 Peptide(beginning=13, end=14, seq='R', modifications=[Modification(name='met_ox', mass=15.9949, count=0)]),
 Peptide(beginning=14, end=21, seq='HGLDNYR', modifications=[Modification(name='met_ox', mass=15.9949, count=0)]),
 Peptide(beginning=21, end=33, seq='GYSLGNWVCAAK', modifications=[Modification(name='met_ox', mass=15.9949, count=0)]),
 Peptide(beginning=33, end=45, seq='FESNFNTQATNR', modifications=[Modification(name='met_ox', mass=15.9949, count=0)]),
 Peptide(beginning=45, end=61, seq='NTDGSTDYGILQINSR', modifications=[Modification(name='met_ox', mass=15.9949, count=0)]),
 Peptide(beginning=61, end=68, seq='WWCNDGR', modifications=[Modification(name='

In [435]:

def within_bounds(reference_mass, measured_mass, ppm_error=10):
    return abs(reference_mass - measured_mass) <= err_margin(reference_mass, ppm_error)


def err_margin(reference_mass, ppm_error=10):
    return (ppm_error / 1e6) * reference_mass


# peptide_masses should be digestion peptides with H2O loss
def precursor_matches(peptides, measurement, ppm_error=10):
    measured_mass = measurement.peptide_mass_estimate
    modifications = {}

    result = []
    b = 0
    reference_mass = mass.calculate_mass(formula="H2O")

    for e in range(len(peptides)):
        reference_mass += peptides[e].zwitterion_mass
        # for mod, count in peptides[e].modifications:
        #    modifications.setdefault(mod, 0)
        #    modifications[mod] += count

        # If the measured mass is higher than our current peptide
        # if reference_mass - err_margin(reference_mass, ppm_error) < measured_mass:
        # difference = reference_mass - measured_mass
        # Here subset sum with tolerance

        # Move bottom tolerance bound below measured_mass
        while reference_mass - err_margin(reference_mass, ppm_error) > measured_mass:
            # for mod, count in peptides[b].modifications:
            #    modifications[mod] -= count
            reference_mass -= peptides[b].zwitterion_mass
            b += 1

        checkpoint_b = b
        checkpoint_mass = reference_mass

        # Check all matches within tolerance with fixed e
        while measured_mass < reference_mass + err_margin(reference_mass, ppm_error):
            if within_bounds(reference_mass, measured_mass, ppm_error):
                result.append({"beginning": b, "end": e + 1, "mc": e - b, "mods": []})

            # for mod, count in peptides[b].modifications:
            #    modifications[mod] -= count
            reference_mass -= peptides[b].zwitterion_mass
            b += 1

        b = checkpoint_b
        reference_mass = checkpoint_mass

    return result


In [125]:
for scan, measurement in measurements.items():
    for match in precursor_matches(peptides, measurement, ppm_error=15):
        peptide = "".join(p.seq for p in peptides[match["beginning"] : match["end"]])
        print(f"{scan}: {peptide}")

846: RHGLDNYR
917: HGLDNYR
1015: HGLDNYR
1070: HGLDNYR
1122: HGLDNYR
1187: HGLDNYR
1259: HGLDNYR
1335: HGLDNYR
1417: HGLDNYR
1474: FESNFNTQATNR
1482: VFGRCELAAAMKR
1505: HGLDNYR
1557: CELAAAMKR
1604: HGLDNYR
1693: HGLDNYR
1783: HGLDNYR
1790: GTDVQAWIRGCR
1902: HGLDNYR
1997: FESNFNTQATNR
2022: FESNFNTQATNR
2030: HGLDNYR
2107: FESNFNTQATNR
2136: FESNFNTQATNR
2149: HGLDNYR
2206: FESNFNTQATNR
2281: FESNFNTQATNR
2284: HGLDNYR
2312: CKGTDVQAWIRGCR
2323: FESNFNTQATNR
2392: FESNFNTQATNR
2393: HGLDNYR
2471: NTDGSTDYGILQINSR
2493: FESNFNTQATNR
2530: FESNFNTQATNR
2532: HGLDNYR
2635: FESNFNTQATNR
2654: CELAAAMKR
2656: FESNFNTQATNR
2658: HGLDNYR
2770: FESNFNTQATNR
2773: HGLDNYR
2774: FESNFNTQATNR
2909: FESNFNTQATNR
2912: HGLDNYR
2914: FESNFNTQATNR
3032: FESNFNTQATNR
3036: HGLDNYR
3037: FESNFNTQATNR
3104: NTDGSTDYGILQINSR
3175: FESNFNTQATNR
3210: FESNFNTQATNR
3211: HGLDNYR
3303: FESNFNTQATNR
3329: WWCNDGR
3338: HGLDNYR
3342: FESNFNTQATNR
3417: FESNFNTQATNR
3456: HGLDNYR
3463: FESNFNTQATNR
3529: FESN

In [436]:

# source: https://stackoverflow.com/questions/23087820/python-subset-sum
def subset(array, num, is_error_ok):
    result = []

    def find(arr, num, path=()):
        if not arr:
            return
        if is_error_ok(num - arr[0]):
            result.append(path + (arr[0],))
        else:
            find(arr[1:], num - arr[0], path + (arr[0],))
            find(arr[1:], num, path)

    find(array, num)

    return list(set(result))


def mod_sum(target, modifications, abs_error):
    masses = tuple(m.mass for m in modifications)

    def helper(t, mods, acc):
        if abs(target - 0) <= abs_error:
            return [acc]
        elif target < 0 - abs_error:
            return []
        else:
            first_nonzero = 0
            while mods[first_nonzero].count == 0:
                first_nonzero += 1
                if first_nonzero >= len(mods):
                    return []
            mods[first_nonzero].count -= 1
            b = helper(t, mods, acc)
            acc2 = acc[:]
            acc2[first_nonzero] += 1
            a = helper(t - masses[first_nonzero], mods, acc2)

            return a + b

    return helper(target, [], [0] * len(modifications))


mods = [Modification("A", 5, 1), Modification("B", 1, 7), Modification("B", 3, 2)]
array = []
for m in mods:
    array += [m.mass] * m.count
subset(array, 10, 0)

TypeError: 'int' object is not callable

In [437]:

from enum import Enum, auto


class State(Enum):
    BEFORE = auto()
    DURING = auto()


# peptide_masses should be digestion peptides with H2O loss
def precursor_matches_2(
    peptides: List[Peptide],
    measurement: PeptideMeasurement,
    cysteine_mod: float,
    jumps: int,
    ppm_error: int = 10,
) -> List[str]:
    target = measurement.peptide_mass_estimate
    h2o = mass.calculate_mass(formula="H2O")
    h2 = mass.calculate_mass(formula="H2")

    result = []

    # TODO: Opravit chybu v hlášení výsledků (moc FP)
    # TODO: Optimize path building
    # TODO: Optimize with dynamic programming if possible
    def go(
        i: int,
        current: float,
        path: Tuple[int, ...],
        state: State,
        jumps: int,
        cysteines_before: int,
        cysteines_now: int,
        min_mod_bound: float,
        max_mod_bound: float,
    ) -> None:
        # Non-bonded cysteines are alkylated, or modified in another way
        cysteines = cysteines_before + cysteines_now
        c_mod_add = cysteine_mod * cysteines

        final_mass = current + c_mod_add

        min_with_c = min_mod_bound + c_mod_add
        lowest = min_with_c - err_margin(min_with_c, ppm_error)

        max_with_c = max_mod_bound + c_mod_add
        highest = max_with_c + err_margin(max_with_c, ppm_error)

        if cysteines_now >= 0 and lowest <= target <= highest:
            seq = peptides[path[0]].seq
            for p in range(1, len(path)):
                if path[p] != path[p - 1] + 1:
                    seq += "+"
                seq += peptides[path[p]].seq

            if within_bounds(final_mass, target, ppm_error):
                result.append(
                    {
                        "sequence": seq,
                        "cysteine_mods": cysteines,
                        "mass": final_mass,
                        "target": target,
                        "error": 1e6 * abs(target - final_mass) / final_mass,
                        "mods": [],
                    }
                )

            mod_array = []
            for p in path:
                for m in peptides[p].modifications:
                    mod_array += [m.mass] * m.count

            diff = target - final_mass

            def is_mod_ok(err):
                superfinal_mass = final_mass + diff - err
                return within_bounds(superfinal_mass, target, ppm_error)

            combinations = subset(
                mod_array,
                diff,
                is_mod_ok,
            )

            for c in combinations:
                result.append(
                    {
                        "sequence": seq,
                        "cysteine_mods": cysteines,
                        "mass": final_mass,
                        "mods": c,
                    }
                )
        elif (
            i == len(peptides)
            or min_mod_bound - err_margin(min_mod_bound, ppm_error) > target
        ):
            # Either we're out of peptides to add
            # Or we're too high and we'll never correct it
            return
        else:
            if state == State.BEFORE:
                # Don't start yet
                go(
                    i + 1,
                    current,
                    path,
                    state.BEFORE,
                    jumps,
                    cysteines_before,
                    cysteines_now,
                    min_mod_bound,
                    max_mod_bound,
                )
            elif state == State.DURING and min(jumps, cysteines) > 0:
                # End this run, begin next one
                go(
                    i,
                    current + h2o - h2,  # +H2O for peptide ends, –H2 for the S-S bond
                    path,
                    state.BEFORE,
                    jumps - 1,
                    cysteines - 1,
                    -1,
                    min_mod_bound + h2o - h2,
                    max_mod_bound + h2o - h2,
                )

            # Take this one, and either begin or continue this run
            go(
                i + 1,
                current + peptides[i].zwitterion_mass,
                path + (i,),
                State.DURING,
                jumps,
                cysteines_before,
                cysteines_now + peptides[i].count("C"),
                min_mod_bound + peptides[i].min_mass,
                max_mod_bound + peptides[i].max_mass,
            )

    go(
        0,
        current=h2o,
        path=(),
        state=State.BEFORE,
        jumps=jumps,
        cysteines_before=0,
        cysteines_now=0,
        min_mod_bound=h2o,
        max_mod_bound=h2o,
    )

    return result


In [388]:
import time

start_time = time.time()
for scan, measurement in measurements.items():
    for match in precursor_matches(peptides, measurement, ppm_error=15):
        peptide = "".join(p.seq for p in peptides[match["beginning"] : match["end"]])
        mods = match["mods"]
        print(f"{scan}: {peptide}")
end_time = time.time()

print(f"This takes {end_time - start_time} seconds")

886: DTHKSEIAHR
893: EKVLTSSAR
910: VLTSSAR
974: YLYEIARR
1038: VTKCCTESLVNR
1063: LKPDPNTLCDEFK
1069: VLTSSAR
1105: QNCDQFEK
1155: LSQKFPK
1285: LSQKFPK
1289: DVCKNYQEAK
1376: FGERALK
1402: LSQKFPK
1429: LCVLHEK
1531: LSQKFPK
1567: LKPDPNTLCDEFK
1577: IETMREK
1580: ATEEQLK
1586: LKPDPNTLCDEFK
1594: QNCDQFEK
1606: ADEKKFWGK
1634: ECCHGDLLECADDRADLAKYICDNQDTISSK
1647: ECCHGDLLECADDRADLAKYICDNQDTISSK
1677: LSQKFPK
1698: ADEKKFWGK
1767: SEIAHRFK
1825: LSQKFPK
1909: IETMREKVLTSSAR
1987: LSQKFPK
1997: EKVLTSSAR
2072: SHCIAEVEK
2089: IETMREKVLTSSAR
2133: LSQKFPK
2145: DLGEEHFK
2161: LKECCDKPLLEK
2170: FKDLGEEHFK
2191: FKDLGEEHFK
2287: DLGEEHFK
2299: LSQKFPK
2307: FKDLGEEHFK
2369: FKDLGEEHFK
2437: FKDLGEEHFK
2438: DDSPDLPK
2445: DLGEEHFK
2469: LVNELTEFAK
2473: AWSVARLSQK
2486: LVNELTEFAK
2527: ALKAWSVAR
2552: FKDLGEEHFK
2568: DDSPDLPK
2582: KQTALVELLKHKPK
2602: LVNELTEFAK
2608: LVTDLTK
2627: AWSVARLSQK
2682: FKDLGEEHFK
2726: LVTDLTK
2792: DDSPDLPK
2812: FKDLGEEHFK
2847: AWSVARLSQK
2854: LVTDL

In [438]:
start_time = time.time()
for scan, measurement in measurements.items():
    for match in precursor_matches_2(
        peptides, measurement, cysteine_mod=57.0214, jumps=1, ppm_error=15
    ):
        seq = match["sequence"]
        mods = match["mods"]
        print(f"{scan}: {seq}")
end_time = time.time()

print(f"This takes {end_time - start_time} seconds")

845: CELAAAMK+GCR
846: RHGLDNYR
848: CELAAAMK+GCR
849: CELAAAMK+CK
852: CELAAAMKR+CK
854: CELAAAMKR+GCR
858: NRCK+GCR
889: CELAAAMK+CK
890: CELAAAMKR+GCR
892: CELAAAMK+CK
893: CELAAAMKR
914: CELAAAMKR
917: HGLDNYR
956: CELAAAMK+GCR
957: CELAAAMK+GCR
964: CELAAAMK+GCR
974: GYSLGNWVCAAK
1007: CELAAAMKR+GCR
1015: HGLDNYR
1033: CELAAAMK+GCR
1036: CELAAAMK+GCR
1038: CELAAAMK+GCR
1057: CELAAAMKR+GCRL
1070: HGLDNYR
1072: CELAAAMKR+GCRL
1086: CELAAAMK+GCR
1093: CELAAAMKR
1097: CELAAAMK
1100: CELAAAMKR
1109: CELAAAMKR+GCRL
1122: HGLDNYR
1128: CELAAAMKR
1138: CELAAAMK+GCR
1157: CELAAAMK
1162: CELAAAMKR+GCRL
1169: CELAAAMKR+GCRL
1171: CELAAAMKR+GCRL
1187: HGLDNYR
1200: CELAAAMKR
1207: CELAAAMK+GCR
1216: CELAAAMK+NRCK
1223: CELAAAMK+GCRL
1224: CELAAAMK
1242: CELAAAMKR+GCRL
1259: HGLDNYR
1265: CELAAAMK+GCRL
1269: CELAAAMKR
1276: CELAAAMK+GCR
1281: CELAAAMKR+GCRL
1287: CELAAAMK+GCRL
1296: CELAAAMK
1303: WWCNDGR+CK
1310: CELAAAMKR+GCRL
1324: WWCNDGR+CK
1332: CELAAAMKR
1335: HGLDNYR
1336: CELAAAMK+GCR

In [433]:
for sc in [1093, 1097]:
    print(sc)
    for p in precursor_matches_2(
        peptides, measurements[sc], cysteine_mod=57.0214, jumps=1, ppm_error=15
    ):
        err = 1e6 * abs(measurements[sc].peptide_mass_estimate - p["mass"]) / p["mass"]
        print(f"{round(err, 2)}ppm", p)

1093
CELAAAMKR 15.990005402318047
[15.9949] 15.990005402318047 ()
1048.51570683823 1064.51060683823 1064.505712240548 -0.004894597681868618 0.01596765910257345
15250.13ppm {'sequence': 'CELAAAMKR', 'cysteine_mods': 1, 'mass': 1048.51570683823, 'mods': (15.9949,)}
1097
CELAAAMK 15.995169160291994
[15.9949] 15.995169160291994 ()
892.41459581463 908.4094958146301 908.409764974922 0.00026916029196399904 0.01362614243721945
17923.47ppm {'sequence': 'CELAAAMK', 'cysteine_mods': 1, 'mass': 892.41459581463, 'mods': (15.9949,)}


In [431]:
subset([15.9949], 15.990005402318047, lambda err: print(err))

[15.9949] 15.990005402318047 ()
-0.004894597681952106
[] -0.004894597681952106 (15.9949,)
[] 15.990005402318047 ()


[]